In [1]:
# # Load the autoreload extension
# %load_ext autoreload

# # Autoreload reloads modules before executing code
# # 0: disable
# # 1: reload modules imported with %aimport
# # 2: reload all modules, except those excluded by %aimport
# %autoreload 2
# !pip install rapid_models

In [2]:
import numpy as np
import scipy as sp
import plotly
import plotly.graph_objs as go

# Import rapid_models DOE packages
import rapid_models.doe as doe


# Convex hull filter

In [3]:
n_small=200
n_large=5000

In [4]:
# Create small lhs to identify points which in_hull method should use as hull basias
lhs = doe.lhs_with_bounds(nDim=2, nSamples=n_small, LBs=[0,0], UBs=[10,5], random_state=42)


In [5]:
# Create functions of upper and lower bounds (LBs and UBs)
f_ub1 = lambda x: 1/3*x + 2
f_ub2 = lambda x: -1/2*x + 6
f_lb1 = lambda x: -2*x + 6
f_lb2 = lambda x: 1/10*x + 0

x=np.linspace(0,10,21)

ub1=f_ub1(x)
ub2=f_ub2(x)
lb1=f_lb1(x)
lb2=f_lb2(x)

In [6]:
# Filter out lhs lower than UBs and larger than LBs
flt_def = (lhs[:,1]<f_ub1(lhs[:,0])) & (lhs[:,1]<f_ub2(lhs[:,0])) & (lhs[:,1]>f_lb1(lhs[:,0])) & (lhs[:,1]>f_lb2(lhs[:,0]))

In [7]:
# Create larger LHS to test in_hull method on
lhs2 = doe.lhs_with_bounds(nDim=2, nSamples=n_large, LBs=[0,0], UBs=[10,5], random_state=47)

In [8]:
flt_inhull=doe.in_hull(lhs2, lhs[flt_def])

In [9]:
f_inside_circ = lambda x,y,x0,y0,r: (x-x0)**2 + (y-y0)**2 < r**2

x0,y0,r=3.5,2.4,2.1

t=np.linspace(0,2*np.pi, 20)

x_circ = x0 + r*np.cos(t)
y_circ = y0 + r*np.sin(t)

flt_def_circ = [f_inside_circ(x,y,x0,y0,r) for x,y in lhs]

flt_inhull_circ=doe.in_hull(lhs2, lhs[flt_def_circ])

# Sample N using kmeans clustering

In [10]:
kms = doe.kmeans_sample(points=lhs2[flt_inhull], N=8, values=None, type="center", random_state=42)

kms_circ = doe.kmeans_sample(points=lhs2[flt_inhull_circ], N=8, values=None, type="center", random_state=42)

# Plot

In [13]:
# Plot all
fig = go.Figure()

fig.add_scatter(x=lhs[:,0], y=lhs[:,1], name='lhs',
  showlegend=True, mode='markers',
  marker={'color':'blue','size':5},
)
fig.add_scatter(x=lhs[flt_def,0], y=lhs[flt_def,1], name='pts_def_hull',
  showlegend=True, mode='markers',
  marker={'color':'red','size':5},
)


q=0
for k,ys in {'ub1':ub1, 'ub2':ub2, 'lb1':lb1, 'lb2':lb2}.items():
  fig.add_scatter(x=x, y=ys, name=k,
    showlegend=True, mode='lines',
    
  )

# plot samples and samples in hull
fig.add_scatter(x=lhs2[:,0], y=lhs2[:,1], name='lhs2',
  showlegend=True, mode='markers',
  marker={'color':'black','size':2},
)
fig.add_scatter(x=lhs2[flt_inhull,0], y=lhs2[flt_inhull,1], name='in_hull',
  showlegend=True, mode='markers',
  marker={'color':'orange','size':3},
)
# plot kmeans samples
fig.add_scatter(x=kms[:,0], y=kms[:,1], name='kms',
  showlegend=True, mode='markers',
  marker={'symbol':'x', 'color':'black','size':7},
)


fig.update_layout(height=500, width=900,
  xaxis={'range':[0,10], 'tick0':0, 'dtick':1, 'showgrid':True, 'gridcolor':'lightgrey', 'zeroline':True, 'zerolinecolor':'gray'},
  yaxis={'range':[0,5], 'tick0':0, 'dtick':1, 'showgrid':True, 'gridcolor':'lightgrey', 'zeroline':True, 'zerolinecolor':'gray'},
  plot_bgcolor='rgba(0,0,0,0)'
)

In [14]:
# Plot all
fig = go.Figure()

fig.add_scatter(x=lhs[:,0], y=lhs[:,1], name='lhs',
  showlegend=True, mode='markers',
  marker={'color':'blue','size':5},
)
fig.add_scatter(x=lhs[flt_def_circ,0], y=lhs[flt_def_circ,1], name='pts_def_circ_hull',
  showlegend=True, mode='markers',
  marker={'color':'red','size':5},
)

fig.add_scatter(x=x_circ, y=y_circ, name='circel',
  showlegend=True, mode='lines',
  line={'color':'magenta'}
)

# plot samples and samples in hull
fig.add_scatter(x=lhs2[:,0], y=lhs2[:,1], name='lhs2',
  showlegend=True, mode='markers',
  marker={'color':'black','size':2},
)
fig.add_scatter(x=lhs2[flt_inhull_circ,0], y=lhs2[flt_inhull_circ,1], name='in_hull',
  showlegend=True, mode='markers',
  marker={'color':'orange','size':3},
)
# plot kmeans samples
fig.add_scatter(x=kms_circ[:,0], y=kms_circ[:,1], name='kms',
  showlegend=True, mode='markers',
  marker={'symbol':'x', 'color':'black','size':7},
)

fig.update_layout(height=500, width=900,
  xaxis={'range':[0,10], 'tick0':0, 'dtick':1, 'showgrid':True, 'gridcolor':'lightgrey', 'zeroline':True, 'zerolinecolor':'gray'},
  yaxis={'range':[0,5], 'tick0':0, 'dtick':1, 'showgrid':True, 'gridcolor':'lightgrey', 'zeroline':True, 'zerolinecolor':'gray'},
  plot_bgcolor='rgba(0,0,0,0)'
)